In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats
from bs4 import BeautifulSoup as bs
from secrets import token1 


In [ ]:
secret = token1

In [2]:
#scrape webpage

def get_spy():
    
    url = 'https://www.slickcharts.com/sp500'

    request = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})

    soup = bs(request.text, "lxml")

    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df =pd.read_html(str(stats))[0]

    df['% Chg'] = df['% Chg'].str.strip('()-%')

    df['% Chg'] = pd.to_numeric(df['% Chg'])

    df['Chg'] = pd.to_numeric(df['Chg'])

    return df


In [3]:
df1 = pd.DataFrame(get_spy()['Symbol'])
df1.rename(columns = {'Symbol':'Ticker'}, inplace = True)
df1

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,GOOGL
4,GOOG
...,...
499,PVH
500,FOX
501,DISH
502,NWS


In [4]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_groups = list(chunks(df1['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
    

In [6]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={secret}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

C:\Users\zvern\AppData\Local\Temp\ipykernel_18916\2502194936.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18916\2502194936.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18916\2502194936.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_18916\2502194936.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ip

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,141.66,N/A,0.070313,N/A,-0.194186,N/A,-0.188026,N/A,0.008113,N/A,N/A
1,MSFT,267.7,N/A,0.010112,N/A,-0.200155,N/A,-0.11848,N/A,0.019732,N/A,N/A
2,AMZN,116.67,N/A,-0.315235,N/A,-0.31922,N/A,-0.293212,N/A,0.090705,N/A,N/A
3,GOOGL,2359.5,N/A,-0.037006,N/A,-0.196993,N/A,-0.167273,N/A,0.115023,N/A,N/A
4,GOOG,2370.76,N/A,-0.066593,N/A,-0.1944,N/A,-0.162403,N/A,0.119979,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,PVH,63.35,N/A,-0.42805,N/A,-0.367936,N/A,-0.242911,N/A,-0.035553,N/A,N/A
500,FOX,30.35,N/A,-0.154806,N/A,-0.114993,N/A,-0.197515,N/A,-0.01365,N/A,N/A
501,DISH,18.85,N/A,-0.555529,N/A,-0.426529,N/A,-0.390165,N/A,-0.141621,N/A,N/A
502,NWS,16.08,N/A,-0.329021,N/A,-0.283582,N/A,-0.299346,N/A,-0.062391,N/A,N/A


In [7]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.742063
1      0.642857
2      0.128968
3       0.56746
4      0.525794
         ...   
499    0.049603
500    0.353175
501    0.019841
502    0.121032
503    0.436508
Name: One-Year Return Percentile, Length: 504, dtype: object
0      0.404762
1      0.386905
2      0.136905
3      0.392857
4      0.402778
         ...   
499    0.075397
500    0.569444
501     0.03373
502    0.202381
503    0.559524
Name: Six-Month Return Percentile, Length: 504, dtype: object
0      0.271825
1      0.519841
2      0.059524
3      0.337302
4      0.355159
         ...   
499    0.136905
500    0.244048
501    0.019841
502    0.053571
503    0.515873
Name: Three-Month Return Percentile, Length: 504, dtype: object
0      0.736111
1      0.795635
2      0.948413
3      0.964286
4      0.970238
         ...   
499    0.505952
500    0.621032
501    0.079365
502    0.325397
503    0.886905
Name: One-Month Return Percentile, Length: 504, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,141.66,N/A,0.070313,0.742063,-0.194186,0.404762,-0.188026,0.271825,0.008113,0.736111,N/A
1,MSFT,267.7,N/A,0.010112,0.642857,-0.200155,0.386905,-0.11848,0.519841,0.019732,0.795635,N/A
2,AMZN,116.67,N/A,-0.315235,0.128968,-0.31922,0.136905,-0.293212,0.059524,0.090705,0.948413,N/A
3,GOOGL,2359.5,N/A,-0.037006,0.56746,-0.196993,0.392857,-0.167273,0.337302,0.115023,0.964286,N/A
4,GOOG,2370.76,N/A,-0.066593,0.525794,-0.1944,0.402778,-0.162403,0.355159,0.119979,0.970238,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
499,PVH,63.35,N/A,-0.42805,0.049603,-0.367936,0.075397,-0.242911,0.136905,-0.035553,0.505952,N/A
500,FOX,30.35,N/A,-0.154806,0.353175,-0.114993,0.569444,-0.197515,0.244048,-0.01365,0.621032,N/A
501,DISH,18.85,N/A,-0.555529,0.019841,-0.426529,0.03373,-0.390165,0.019841,-0.141621,0.079365,N/A
502,NWS,16.08,N/A,-0.329021,0.121032,-0.283582,0.202381,-0.299346,0.053571,-0.062391,0.325397,N/A


In [8]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [9]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [11]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Enter the value of your portfolio')
  global val

  try:
    val = float(portfolio_size)
    print(portfolio_size)
  
  except ValueError:
    print('Please enter a number \n Try again')

In [12]:
portfolio_input()

Enter the value of your portfolio 10000


10000


In [14]:

position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe.head(10)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,141.66,1,0.070313,0.742063,-0.194186,0.404762,-0.188026,0.271825,0.008113,0.736111,0.53869
1,MSFT,267.7,0,0.010112,0.642857,-0.200155,0.386905,-0.11848,0.519841,0.019732,0.795635,0.58631
2,AMZN,116.67,1,-0.315235,0.128968,-0.31922,0.136905,-0.293212,0.059524,0.090705,0.948413,0.318452
3,GOOGL,2359.5,0,-0.037006,0.56746,-0.196993,0.392857,-0.167273,0.337302,0.115023,0.964286,0.565476
4,GOOG,2370.76,0,-0.066593,0.525794,-0.1944,0.402778,-0.162403,0.355159,0.119979,0.970238,0.563492
5,TSLA,737.12,0,0.097117,0.787698,-0.309166,0.14881,-0.27064,0.095238,0.118883,0.968254,0.5
6,BRK.B,278.28,0,0,0.625992,0,0.762897,0,0.852183,0,0.684524,0.731399
7,JNJ,182.29,1,0.138498,0.853175,0.097409,0.865079,0.036974,0.918651,0.014865,0.77381,0.852679
8,UNH,495.64,0,0.24031,0.928571,0.007064,0.771825,-0.030451,0.78373,-0.00137,0.674603,0.789683
9,NVDA,171.26,1,-0.099487,0.468254,-0.421979,0.039683,-0.381424,0.02381,0.009109,0.746032,0.319444


In [15]:
writer = pd.ExcelWriter('HQM.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='HQM', index = False)

In [16]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['HQM'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['HQM'].write(f'{column}1', column_formats[column][0], string_template)

In [20]:
writer.save()